In [1]:
# ROOT::VecOps::RVec<Int_t>
# ROOT::VecOps::RVec<Int_t>
# ROOT::VecOps::RVec<Float_t>
# ROOT::VecOps::RVec<Float_t>
# UInt_t

# Funciones definitivas

In [2]:
import os, sys, math
import ROOT
import multiprocessing

Welcome to JupyROOT 6.24/06


#### Taggeado usando $\Delta R$

In [3]:
# Función para buscar el índice del jet que busco

ROOT.gInterpreter.Declare('''
using Vbool = ROOT::VecOps::RVec<bool>;
using Vfloat = ROOT::VecOps::RVec<float>;
using Vint = ROOT::VecOps::RVec<int>;

Vbool is_prompt_dsa(
    Vfloat  dsamu_d0_pv
) {

    // Cortes en d0 para DSA
        float cut_dsa = 1;    //cm

    // Cantidad de muones DSA
        unsigned int n_dsa = dsamu_d0_pv.size();

    // Vector booleano de la máscara
        bool is_prompt = false;
        Vbool mask;

    // Miramos si alguno pasa el corte
        for (unsigned int i=0; i<n_dsa; ++i) {
            if (dsamu_d0_pv[i] <= cut_dsa) {
                is_prompt = is_prompt || true;
            }else{
                is_prompt = is_prompt || false;
            }
        }

        for (unsigned int i=0; i<n_dsa; ++i) {
            mask.push_back(is_prompt);
        }

    return mask;
};

''')

True

#### Taggeado usando partonFlavour

In [4]:
ROOT.gInterpreter.Declare("""
    using Vbool = ROOT::RVec<bool>;
    using Vfloat = ROOT::RVec<float>;
    using Vint = ROOT::RVec<int>;
    using VfloatVec = ROOT::VecOps::RVec<Float_t>;
    using VintVec = ROOT::VecOps::RVec<Int_t>;
    
    VintVec index_bjet_2(VintVec GenJet_partonFlavour, VintVec Jet_genJetIdx) {
    
    Int_t index_b;
    Int_t index_antib;
    
    Int_t b = 5;
    Int_t antib = -5;
    Size_t dim_size =  GenJet_partonFlavour.size();
    unsigned int dim_uint = static_cast<unsigned int>(dim_size);
    
    // Ver si hay varios con el mismo tipo (5 o -5), y etiquetar con -10
    
    Int_t cuentas_b = 0;
    Int_t cuentas_antib = 0;
    for (unsigned int i=0; i<dim_uint; ++i) {
        if (GenJet_partonFlavour[i] == b) {
            cuentas_b = cuentas_b + 1;
        } 
        if (GenJet_partonFlavour[i] == antib) {
            cuentas_antib = cuentas_antib + 1;
        }
    }
    if (cuentas_b >= 2) {
        index_b = -10;
    } 
    if (cuentas_antib >= 2) {
        index_antib = -10;
    }    
    
    // Ver si no hay ninguno de ese tipo (5 o -5), y etiquetar con -11
    
    if (cuentas_b == 0) {
        index_b = -11;
    } 
    if (cuentas_antib == 0) {
        index_antib = -11;
    }    
    
    // Asignar los índices correspondientes
    
    Size_t dim_idx_size =  Jet_genJetIdx.size();
    unsigned int dim_idx = static_cast<unsigned int>(dim_idx_size);
    
    if (cuentas_b ==1 ) {
        for (unsigned int i=0; i<dim_uint; ++i) {
            if (GenJet_partonFlavour[i] == b) {
                if (i >= dim_idx) {
                    index_b = -12; // Se ha perdido el jet
                } else {
                    if (Jet_genJetIdx[i] < 0 || Jet_genJetIdx[i] >= dim_idx) {
                        index_b = -13; // Indice negativo
                    } else {
                        index_b = Jet_genJetIdx[i];
                    }
                }
            }
        }
    }
    if (cuentas_antib ==1 ) {
        for (unsigned int i=0; i<dim_uint; ++i) {
            if (GenJet_partonFlavour[i] == antib) {
                if (i >= dim_idx) {
                    index_antib = -12; // Se ha perdido el jet
                } else {
                    if (Jet_genJetIdx[i] < 0 || Jet_genJetIdx[i] >= dim_idx) {
                        index_antib = -13; // Indice negativo
                    } else {
                        index_antib = Jet_genJetIdx[i];
                    }
                }
            }
        }
    }
    
    VintVec indexes = {index_b, index_antib};
    return indexes;
    };
""")

True

#### Taggedo usando Jet genIndex

In [3]:
ROOT.gInterpreter.Declare("""
    using Vbool = ROOT::RVec<bool>;
    using Vfloat = ROOT::RVec<float>;
    using Vint = ROOT::RVec<int>;
    using VfloatVec = ROOT::VecOps::RVec<Float_t>;
    using VintVec = ROOT::VecOps::RVec<Int_t>;
    
    VintVec index_bjet_2(VintVec GenJet_partonFlavour, VintVec Jet_genJetIdx) {
    
    Int_t index_b;
    Int_t index_antib;
    
    Int_t b = 5;
    Int_t antib = -5;
    
    // Dimensiones de los vectores
    
    Size_t dim_size =  GenJet_partonFlavour.size();
    unsigned int dim_uint = static_cast<unsigned int>(dim_size);
    Size_t dim_idx_size =  Jet_genJetIdx.size();
    unsigned int dim_idx = static_cast<unsigned int>(dim_idx_size);
    
    // Dimensión menor
    
    unsigned int dim_min = std::min(dim_uint, dim_idx);
    
    // Voy recorriendo Jet_genJetIdx
    // Tengo que ignorar cuando Jet_genJetIdx es -1 (jet reconstruido que no estaba a nivek generador)
    // Miro las cuentas de b y antib
    
    Int_t cuentas_b = 0;
    Int_t cuentas_antib = 0;
    
    for (unsigned int i=0; i<dim_min; ++i) {
        if (GenJet_partonFlavour[i] == b && Jet_genJetIdx[i] != -1 && Jet_genJetIdx[i] < dim_idx) {
            cuentas_b = cuentas_b + 1;
        } 
        if (GenJet_partonFlavour[i] == antib && Jet_genJetIdx[i] != -1 && Jet_genJetIdx[i] < dim_idx) {
            cuentas_antib = cuentas_antib + 1;
        }
    }
    
    // Casos en los que hay más de 2 jets de b quarks
    
    if (cuentas_b >= 2) {
        index_b = -10;
    } 
    if (cuentas_antib >= 2) {
        index_antib = -10;
    }   
    
    // Ver si no hay ninguno de ese tipo (5 o -5), y etiquetar con -11
    
    if (cuentas_b == 0) {
        index_b = -11;
    } 
    if (cuentas_antib == 0) {
        index_antib = -11;
    } 
    
    
    // Asigno los indices
    
    if (cuentas_b ==1 ) {
        for (unsigned int i=0; i<dim_min; ++i) {
            if (GenJet_partonFlavour[i] == b && Jet_genJetIdx[i] != -1 && Jet_genJetIdx[i] < dim_idx) {
                index_b = Jet_genJetIdx[i];
            }   
        }
    }
    if (cuentas_antib ==1 ) {
        for (unsigned int i=0; i<dim_min; ++i) {
            if (GenJet_partonFlavour[i] == antib && Jet_genJetIdx[i] != -1 && Jet_genJetIdx[i] < dim_idx) {
                index_antib = Jet_genJetIdx[i];
            }   
        }
    }
    
    VintVec indexes = {index_b, index_antib};
    return indexes;
    };
""")


True

In [4]:
# Odeno los índices en pt

ROOT.gInterpreter.Declare("""
    using Vbool = ROOT::RVec<bool>;
    using Vfloat = ROOT::RVec<float>;
    using VfloatVec = ROOT::VecOps::RVec<Float_t>;
    using VintVec = ROOT::VecOps::RVec<Int_t>;
    
    VintVec index_pt(Int_t idx_genbjet1, Int_t idx_genbjet2, VfloatVec Jet_pt) {
    
    // Miro cual es el pt de cada jet
    Float_t pt_1 = Jet_pt[idx_genbjet1];
    Float_t pt_2 = Jet_pt[idx_genbjet2];
    
    // Comparo ambos y asigno
    Int_t idx_genbjet1_pt;
    Int_t idx_genbjet2_pt;
    
        if (pt_1 > pt_2) {
            idx_genbjet1_pt = idx_genbjet1;
            idx_genbjet2_pt = idx_genbjet2;
        } else {
            idx_genbjet1_pt = idx_genbjet2;
            idx_genbjet2_pt = idx_genbjet1;
        }
    
    // Creo el output
    VintVec indexes_order = {idx_genbjet1_pt, idx_genbjet2_pt};
        
        return indexes_order;
    };
""")

True

In [5]:
# Compruebo qué índices coinciden con los de HHbtag

ROOT.gInterpreter.Declare("""
    using Vbool = ROOT::RVec<bool>;
    using Vfloat = ROOT::RVec<float>;
    using VfloatVec = ROOT::VecOps::RVec<Float_t>;
    using VintVec = ROOT::VecOps::RVec<Int_t>;
    
    Int_t index_coin_hhbtag(Int_t idx_genbjet1, Int_t idx_genbjet2, Int_t bjet1_JetIdx, Int_t bjet2_JetIdx) {
    
    // 1: Coinciden los indices, -1 no
    Int_t is_index_coin;
    
    // Primer caso
        if (idx_genbjet1 == bjet1_JetIdx) {
            if (idx_genbjet2 == bjet2_JetIdx) {
                is_index_coin = 1;
            } else {
                is_index_coin = -1;
            }
    // Segundo caso
        } else if (idx_genbjet1 == bjet2_JetIdx) {
            if (idx_genbjet2 == bjet1_JetIdx) {
                is_index_coin = 1;
            } else {
                is_index_coin = -1;
            }
    // Si ninguno coincide
        } else {
            is_index_coin = -1;
        }
        
        return is_index_coin;
    };
""")

True

In [6]:
# Compruebo qué índices coinciden con los de deepflavour

ROOT.gInterpreter.Declare("""
    using Vbool = ROOT::RVec<bool>;
    using Vfloat = ROOT::RVec<float>;
    using VfloatVec = ROOT::VecOps::RVec<Float_t>;
    using VintVec = ROOT::VecOps::RVec<Int_t>;
    
    Int_t index_coin_vec(Int_t idx_genbjet1, Int_t idx_genbjet2, VfloatVec Jet_btagDeepFlavB) {
    
    // ordeno adecuadamente el vector y selecciono las posiciones de los altos
    
    VfloatVec deepflavor_orden = ROOT::VecOps::Argsort(Jet_btagDeepFlavB);
    Size_t size_deepf = Jet_btagDeepFlavB.size();
    unsigned int dim_deepf_unit = static_cast<unsigned int>(size_deepf);
    
    unsigned int deepf_index_1 = deepflavor_orden[dim_deepf_unit - 1];
    unsigned int deepf_index_2 = deepflavor_orden[dim_deepf_unit - 2];
    
    // 1: Coinciden los indices, -1 no
    Int_t is_index_coin_deepf;
    
    // Primer caso
        if (idx_genbjet1 == deepf_index_1) {
            if (idx_genbjet2 == deepf_index_2) {
                is_index_coin_deepf = 1;
            } else {
                is_index_coin_deepf = -1;
            }
    // Segundo caso
        } else if (idx_genbjet1 == deepf_index_2) {
            if (idx_genbjet2 == deepf_index_1) {
                is_index_coin_deepf = 1;
            } else {
                is_index_coin_deepf = -1;
            }
    // Si ninguno coincide
        } else {
            is_index_coin_deepf = -1;
        }
        
        return is_index_coin_deepf;
    };
""")

True

# USANDO $\Delta R$

In [ ]:
#! /usr/bin/env python3
#
# 1) Setup ROOT with: 
#  source /cvmfs/fcc.cern.ch/sw/latest/setup.sh 
# 2) Run this script as:
#   "python -i [python_script]"
#

import os, sys, math
import ROOT
import multiprocessing
import numpy as np

# Enable multi-threading (parallel processing, do not use more than 8 cores)
maxcpus = multiprocessing.cpu_count()
ROOT.EnableImplicitMT(min(maxcpus,8))
usecpus = max(ROOT.GetThreadPoolSize(),1)
#print("ROOT: %s CPUs used out of %s available ..." % (usecpus, maxcpus))

#Parametros configurables

# Valores :
#btag: tight=0.7264, medium=0.2770, loose=0.0494)
#deeptau:
#    vsjet=(VVVLoose=1, VVLoose=3, VLoose=7, Loose=15, Medium=31,
#        Tight=63, VTight=127, VVTight=255),
#    vse=(VVVLoose=1, VVLoose=3, VLoose=7, Loose=15, Medium=31,
#        Tight=63, VTight=127, VVTight=255),
#    vsmu=(VLoose=1, Loose=3, Medium=7, Tight=15),
#)

# Varible pairType
#     = 0 mutau
#     = 1 etau
#     = 2 tautau

TauIsoThreshold= 5 # el quinto bit, correspondiente a 31 # Seria mejor usar un Diccionario 

# Archivos de entrada
sampleName=["ggf_sm","tt_dl","tt_sl","tt_fh","dy"]
# cutName=["LeptonID","2MedTag","2CandTag","Ellip"]
xsec=[0.03105,88.29,365.34,377.96,6077.22]
NMC=[400000,145020000,476408000,334206000,195510810]
#WeightsMC=[400000,10412949936.433165,143353855628.27905,104501672317.91554,3323473259844.586]
WeightsMC=[400000,10412949936,143353855628,104501672318,3323473259845]

nProcesses=len(sampleName) # numero de procesos
# nCuts=len(cutName)

# Archivos de entrada con el ultimo algoritmo de Taus que tengo (a fecha de 22/02/23)
path='/nfs/cms/cepeda/cmt/PreprocessRDF/ul_2018_v10_cie/'
selection='/cat_base_selection/prodTestV10_10Jan/'


# Cada rootfile tiene un 'tree' llamado events que vamos a
# convertir en un dataframe y guardar en una lista de dataframes. 

print ("Reading Trees")
df = {}


for p in sampleName: # list of strings of the processes
    dir_path=path+p+selection
    names = ROOT.std.vector('string')()
    nfiles=len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))])
    for i in range(0,nfiles):
        names.push_back(dir_path+'data_{}.root'.format(i))
        df[p] = ROOT.RDataFrame("Events",(names))

processes = list(df.keys())

print(processes)

# Ahora hacemos la seleccion y definimos variables nuevas 
allCutsReport = {}

#count_HHbtag = {}
#count_Deepflav = {}
#count_AK4 = {}

for i in range(nProcesses):
    p=processes[i] # key correspondiente

    norm=xsec[i]/WeightsMC[i] # normalizacion sin luminosidad

    df[p] = df[p].Define("myweight","genWeight* %s" %norm) # defino el peso del proceso en el df

    sel_lepton_cross = "Muon_pt[Muon_pt > 20].size() > 0 && Tau_pt[Tau_pt > 27].size() > 0 && Jet_pt[Jet_pt > 20].size() > 1"
    # seleccion basica
    df[p] = df[p].Filter(sel_lepton_cross,"MuTau selection")
    df[p] = df[p].Filter("pairType==0", "Type 0")
    df[p] = df[p].Filter("isOS==1", "Opposite Sign")
    df[p] = df[p].Filter("dau2_idDeepTau2017v2p1VSjet >= %s" %TauIsoThreshold,"Isolated Tau") # Tauisothreshold está en 5 para mutau
    df[p] = df[p].Filter("abs(Muon_dxy.at(dau1_index)) < 0.045","Dist dau1 xy")
    df[p] = df[p].Filter("abs(Muon_dz.at(dau1_index)) < 0.2","Dist dau1 z")
    df[p] = df[p].Filter("abs(Tau_dz.at(dau2_index)) < 0.2","Dist dau2 z")
    df[p] = df[p].Filter("((Htt_svfit_mass - 129.) * (Htt_svfit_mass - 129.)/(53. * 53.)+ (Hbb_mass - 169.) * (Hbb_mass - 169.) / (145. * 145.)) < 1","Ellipsis")
    
    # prints y reports
    print("CutFlow for process", sampleName[i])
    #allCutsReport[p]=df[p].Report()
    #allCutsReport[p].Print()
    #print("HHbtag",df_NOleptonID[p].Count("Jet_HHbtag[Jet_HHbtag > 0]"))
    #print("HH_DeepFlav",df_NOleptonID[p].Count("Jet_btagDeepFlavB"))
    #print("HHbtag",df_NOleptonID[p].Count("Jet_particleNetAK4_B[Jet_particleNetAK4_B > 0]"))

    
for i in range(nProcesses):
    p=processes[i]
    # Para histogramas 2D
    df[p] = df[p].Define("Jet_DeepFlavB_cand1","Jet_btagDeepFlavB.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_DeepFlavB_cand2","Jet_btagDeepFlavB.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_particleNetAK4_B_cand1","Jet_particleNetAK4_B.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_particleNetAK4_B_cand2","Jet_particleNetAK4_B.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_cand1","Jet_HHbtag.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_cand2","Jet_HHbtag.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_total","Jet_HHbtag.at(bjet1_JetIdx) + Jet_HHbtag.at(bjet2_JetIdx)")

mycall_b1 = "index_bjet(5, GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"
mycall_b2 = "index_bjet(-5, GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet1",mycall_b1)
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2",mycall_b2)

mycall_order_pt = "index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet1_pt","index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)[0]")
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2_pt","index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)[1]")

mycall_coinc = "index_coin(idx_genbjet1, idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_coinc",mycall_coinc)

df["ggf_sm"] = df["ggf_sm"].Define("Jet_DeepFlavB_gen1","Jet_btagDeepFlavB.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_DeepFlavB_gen2","Jet_btagDeepFlavB.at(idx_genbjet2_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_particleNetAK4_B_gen1","Jet_particleNetAK4_B.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_particleNetAK4_B_gen2","Jet_particleNetAK4_B.at(idx_genbjet2_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_HHbtag_gen1","Jet_HHbtag.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_HHbtag_gen2","Jet_HHbtag.at(idx_genbjet2_pt)")

#mycall4 = "gen_phi2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_phi)"
#mycall4 = "gen_eta2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta)"
#mycall4 = "gen_index2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2",mycall4)
#mycall6 = "index1_coinc2(idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx2_coin",mycall6)

#mycall_tot = "index_tot(idx_genbjet1, idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx_tot",mycall_tot)
    
DeepFlav_bjet1 = {}
DeepFlav_bjet2 = {}
AK4_bjet1 = {}
AK4_bjet2 = {}
HHbtag_bjet1 = {}
HHbtag_bjet2 = {}
HHbtag_total = {}
DeepFlav_2D = {}
AK4_2D = {}
HHbtag_2D = {}

for i in range(nProcesses):
    p=processes[i]
    DeepFlav_bjet1[p] = df[p].Histo1D(("DeepFlav_bjet1_{}".format(p), "DeepFlavourB Jet1;DeepFlavB jet1;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_cand1","myweight")
    DeepFlav_bjet2[p] = df[p].Histo1D(("DeepFlav_bjet2_{}".format(p), "DeepFlavourB Jet2;DeepFlavB jet2;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_cand2","myweight")
    AK4_bjet1[p] = df[p].Histo1D(("AK4_bjet1_{}".format(p), "particleNetAK4 Jet1;particleNetAK4 jet1;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_cand1","myweight")
    AK4_bjet2[p] = df[p].Histo1D(("AK4_bjet2_{}".format(p), "particleNetAK4 Jet2;particleNetAK4 jet2;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_cand2","myweight")
    HHbtag_bjet1[p] = df[p].Histo1D(("HHbtag_bjet1_{}".format(p), "HHbtag Jet1;HHbtag jet1;N_{Events}",60,0, 2.2), "Jet_HHbtag_cand1","myweight")
    HHbtag_bjet2[p] = df[p].Histo1D(("HHbtag_bjet2_{}".format(p), "HHbtag Jet2;HHbtag jet2;N_{Events}",60,0, 2.2), "Jet_HHbtag_cand2","myweight")
    HHbtag_total[p] = df[p].Histo1D(("HHbtag_total_{}".format(p), "HHbtag 2 jets;HHbtag 2 jets;N_{Events}",60,0, 2.2), "Jet_HHbtag_total","myweight")
    
    DeepFlav_2D[p] = df[p].Histo2D(("DeepFlavB_2D_{}".format(p),"Jet btagDeepFlav 2D;DeepFlavB jet1;DeepFlavB jet2",30,0,1.2,30,0,1.2), "Jet_DeepFlavB_cand1", "Jet_DeepFlavB_cand2","myweight")
    AK4_2D[p] = df[p].Histo2D(("AK4_2D_{}".format(p),"Jet particleNetAK4 2D;particleNetAK4 jet1;particleNetAK4 jet2",30,0,1.2,30,0,1.2), "Jet_particleNetAK4_B_cand1", "Jet_particleNetAK4_B_cand2","myweight")
    HHbtag_2D[p] = df[p].Histo2D(("HHbtag_2D_{}".format(p),"Jet HHbtag 2D;HHbtag jet1;HHbtag jet2",60,0,2.2,60,0,2.2), "Jet_HHbtag_cand1", "Jet_HHbtag_cand2","myweight")

    
#Gen_phi1 = df["ggf_sm"].Histo1D(("Gen_phi1","Phi Gen b;Phi Gen b;N_{Events}",30,-5,11),"Genb1_phi","myweight")
#Gen_phi2 = df["ggf_sm"].Histo1D(("Gen_phi2","Phi Gen anti b;Phi Gen anyi b;N_{Events}",30,-5,11),"Genb2_phi","myweight")
Index1 = df["ggf_sm"].Histo1D(("Index_gen1","Index Gen b;Index Gen b;N_{Events}",20,0,20),"idx_genbjet1","myweight")
Index2 = df["ggf_sm"].Histo1D(("Index_gen2","Index Gen anti b;Index Gen anti b;N_{Events}",20,0,20),"idx_genbjet2","myweight")
#Index1_coin = df["ggf_sm"].Histo1D(("Index_gen1_coin","Index 1 coincidence;Index 1 coincidence;N_{Events}",5,-2,3),"idx1_coin","myweight")
#Index2_coin = df["ggf_sm"].Histo1D(("Index_gen2_coin","Index 2 coincidence;Index 2 coincidence;N_{Events}",5,-2,3),"idx2_coin","myweight")
#Index_2D = df["ggf_sm"].Histo2D(("Index_2D","Index bjet 2D;Index jet1;Index jet2",5,-2,3,5,-2,3), "idx1_coin", "idx2_coin","myweight")
Index_tot = df["ggf_sm"].Histo1D(("Index_tot_coin","Indexes coincidence;Indexes coincidence;N_{Events}",5,-2,3),"idx_coinc","myweight")    

DeepFlav_bjet1_gen = df["ggf_sm"].Histo1D(("DeepFlav_bjet1_gen", "DeepFlavourB Jet1;DeepFlavB jet1;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen1","myweight")
DeepFlav_bjet2_gen = df["ggf_sm"].Histo1D(("DeepFlav_bjet2_gen", "DeepFlavourB Jet2;DeepFlavB jet2;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen2","myweight")
AK4_bjet1_gen = df["ggf_sm"].Histo1D(("AK4_bjet1_gen", "particleNetAK4 Jet1;particleNetAK4 jet1;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen1","myweight")
AK4_bjet2_gen = df["ggf_sm"].Histo1D(("AK4_bjet2_gen", "particleNetAK4 Jet2;particleNetAK4 jet2;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen2","myweight")
HHbtag_bjet1_gen = df["ggf_sm"].Histo1D(("HHbtag_bjet1_gen", "HHbtag Jet1;HHbtag jet1;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen1","myweight")
HHbtag_bjet2_gen = df["ggf_sm"].Histo1D(("HHbtag_bjet2_gen", "HHbtag Jet2;HHbtag jet2;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen2","myweight")
DeepFlav_2D_gen = df["ggf_sm"].Histo2D(("DeepFlavB_2D_gen","Jet btagDeepFlav 2D;DeepFlavB jet1;DeepFlavB jet2",30,0,1.2,30,0,1.2), "Jet_DeepFlavB_gen1", "Jet_DeepFlavB_gen2","myweight")
AK4_2D_gen = df["ggf_sm"].Histo2D(("AK4_2D_gen","Jet particleNetAK4 2D;particleNetAK4 jet1;particleNetAK4 jet2",30,0,1.2,30,0,1.2), "Jet_particleNetAK4_B_gen1", "Jet_particleNetAK4_B_gen2","myweight")
HHbtag_2D_gen = df["ggf_sm"].Histo2D(("HHbtag_2D_gen","Jet HHbtag 2D;HHbtag jet1;HHbtag jet2",60,0,2.2,60,0,2.2), "Jet_HHbtag_gen1", "Jet_HHbtag_gen2","myweight")


out = ROOT.TFile("histos_2.root","RECREATE")
out.cd()

for i in range(nProcesses):
    p = processes[i]
    
    DeepFlav_bjet1[p].Write()
    DeepFlav_bjet2[p].Write()
    AK4_bjet1[p].Write()
    AK4_bjet2[p].Write()
    HHbtag_bjet1[p].Write()
    HHbtag_bjet2[p].Write()
    HHbtag_total[p].Write()
    DeepFlav_2D[p].Write()
    AK4_2D[p].Write()
    HHbtag_2D[p].Write()
    
#Gen_phi1.Write()
#Gen_phi2.Write()
Index1.Write()
Index2.Write()
#Index1_coin.Write()
#Index2_coin.Write()
#Index_2D.Write()
Index_tot.Write()

DeepFlav_bjet1_gen.Write()
DeepFlav_bjet2_gen.Write()
AK4_bjet1_gen.Write()
AK4_bjet2_gen.Write()
HHbtag_bjet1_gen.Write()
HHbtag_bjet2_gen.Write()
DeepFlav_2D_gen.Write()
AK4_2D_gen.Write()
HHbtag_2D_gen.Write()

out.Close()
print("Terminado")

Reading Trees
['ggf_sm', 'tt_dl', 'tt_sl', 'tt_fh', 'dy']
CutFlow for process ggf_sm
CutFlow for process tt_dl
CutFlow for process tt_sl
CutFlow for process tt_fh
CutFlow for process dy


In [8]:
print(df["ggf_sm"].GetColumnType("GenPart_pdgId"))
print(df["ggf_sm"].GetColumnType("GenPart_genPartIdxMother"))
print(df["ggf_sm"].GetColumnType("GenPart_phi"))
print(df["ggf_sm"].GetColumnType("Jet_phi"))
print(df["ggf_sm"].GetColumnType("nJet"))
print(df["ggf_sm"].GetColumnType("bjet1_JetIdx"))
print(df["ggf_sm"].GetColumnType("GenJet_partonFlavour"))
print(df["ggf_sm"].GetColumnType("Jet_genJetIdx"))
print("")
#print(df["ggf_sm"].GetColumnType("Genb1_phi"))
#print(df["ggf_sm"].GetColumnType("Dif_abs_phi_1"))
print(df["ggf_sm"].GetColumnType("idx_genbjet1"))
print(df["ggf_sm"].GetColumnType("idx_coinc"))


ROOT::VecOps::RVec<Int_t>
ROOT::VecOps::RVec<Int_t>
ROOT::VecOps::RVec<Float_t>
ROOT::VecOps::RVec<Float_t>
UInt_t
Int_t
ROOT::VecOps::RVec<Int_t>
ROOT::VecOps::RVec<Int_t>

int
int


# USANDO PARTONFLAVOUR

In [7]:
#! /usr/bin/env python3
#
# 1) Setup ROOT with: 
#  source /cvmfs/fcc.cern.ch/sw/latest/setup.sh 
# 2) Run this script as:
#   "python -i [python_script]"
#

import os, sys, math
import ROOT
import multiprocessing
import numpy as np

# Enable multi-threading (parallel processing, do not use more than 8 cores)
maxcpus = multiprocessing.cpu_count()
ROOT.EnableImplicitMT(min(maxcpus,8))
usecpus = max(ROOT.GetThreadPoolSize(),1)
#print("ROOT: %s CPUs used out of %s available ..." % (usecpus, maxcpus))

#Parametros configurables

# Valores :
#btag: tight=0.7264, medium=0.2770, loose=0.0494)
#deeptau:
#    vsjet=(VVVLoose=1, VVLoose=3, VLoose=7, Loose=15, Medium=31,
#        Tight=63, VTight=127, VVTight=255),
#    vse=(VVVLoose=1, VVLoose=3, VLoose=7, Loose=15, Medium=31,
#        Tight=63, VTight=127, VVTight=255),
#    vsmu=(VLoose=1, Loose=3, Medium=7, Tight=15),
#)

# Varible pairType
#     = 0 mutau
#     = 1 etau
#     = 2 tautau

TauIsoThreshold= 5 # el quinto bit, correspondiente a 31 # Seria mejor usar un Diccionario 

# Archivos de entrada
sampleName=["ggf_sm","tt_dl","tt_sl","tt_fh","dy"]
# cutName=["LeptonID","2MedTag","2CandTag","Ellip"]
xsec=[0.03105,88.29,365.34,377.96,6077.22]
NMC=[400000,145020000,476408000,334206000,195510810]
#WeightsMC=[400000,10412949936.433165,143353855628.27905,104501672317.91554,3323473259844.586]
WeightsMC=[400000,10412949936,143353855628,104501672318,3323473259845]

nProcesses=len(sampleName) # numero de procesos
# nCuts=len(cutName)

# Archivos de entrada con el ultimo algoritmo de Taus que tengo (a fecha de 22/02/23)
path='/nfs/cms/cepeda/cmt/PreprocessRDF/ul_2018_v10_cie/'
selection='/cat_base_selection/prodTestV10_10Jan/'


# Cada rootfile tiene un 'tree' llamado events que vamos a
# convertir en un dataframe y guardar en una lista de dataframes. 

print ("Reading Trees")
df = {}


for p in sampleName: # list of strings of the processes
    dir_path=path+p+selection
    names = ROOT.std.vector('string')()
    nfiles=len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))])
    for i in range(0,nfiles):
        names.push_back(dir_path+'data_{}.root'.format(i))
        df[p] = ROOT.RDataFrame("Events",(names))

processes = list(df.keys())

print(processes)

# Ahora hacemos la seleccion y definimos variables nuevas 
allCutsReport = {}

#count_HHbtag = {}
#count_Deepflav = {}
#count_AK4 = {}

for i in range(nProcesses):
    p=processes[i] # key correspondiente

    norm=xsec[i]/WeightsMC[i] # normalizacion sin luminosidad

    df[p] = df[p].Define("myweight","genWeight* %s" %norm) # defino el peso del proceso en el df

    sel_lepton_cross = "Muon_pt[Muon_pt > 21].size() > 0 && Tau_pt[Tau_pt > 32].size() > 0 && Jet_pt[Jet_pt > 20].size() > 1"
    # seleccion basica
    df[p] = df[p].Filter(sel_lepton_cross,"MuTau selection")
    df[p] = df[p].Filter("pairType==0", "Type 0")
    df[p] = df[p].Filter("isOS==1", "Opposite Sign")
    df[p] = df[p].Filter("Muon_eta[abs(Muon_eta < 2.1)].size() > 0", "eta for mu")
    df[p] = df[p].Filter("Tau_eta[abs(Tau_eta < 2.1)].size() > 0", "eta for tau")
    df[p] = df[p].Define("delta_R","ROOT::VecOps::DeltaR(Muon_eta.at(dau1_index), Tau_eta.at(dau2_index), Muon_phi.at(dau1_index),Tau_phi.at(dau2_index))")
    df[p] = df[p].Filter("delta_R > 0.5", "delta R cut")
    df[p] = df[p].Filter("dau2_idDeepTau2017v2p1VSjet >= %s" %TauIsoThreshold,"Isolated Tau") # Tauisothreshold está en 5 para mutau
    df[p] = df[p].Filter("abs(Muon_dxy.at(dau1_index)) < 0.045","Dist dau1 xy")
    df[p] = df[p].Filter("abs(Muon_dz.at(dau1_index)) < 0.2","Dist dau1 z")
    df[p] = df[p].Filter("abs(Tau_dz.at(dau2_index)) < 0.2","Dist dau2 z")
    #df[p] = df[p].Filter("((Htt_svfit_mass - 129.) * (Htt_svfit_mass - 129.)/(53. * 53.)+ (Hbb_mass - 169.) * (Hbb_mass - 169.) / (145. * 145.)) < 1","Ellipsis")
    
    # prints y reports
    print("CutFlow for process", sampleName[i])
    #allCutsReport[p]=df[p].Report()
    #allCutsReport[p].Print()
    #print("HHbtag",df_NOleptonID[p].Count("Jet_HHbtag[Jet_HHbtag > 0]"))
    #print("HH_DeepFlav",df_NOleptonID[p].Count("Jet_btagDeepFlavB"))
    #print("HHbtag",df_NOleptonID[p].Count("Jet_particleNetAK4_B[Jet_particleNetAK4_B > 0]"))

    
for i in range(nProcesses):
    p=processes[i]
    # Para histogramas 2D
    df[p] = df[p].Define("Jet_DeepFlavB_cand1","Jet_btagDeepFlavB.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_DeepFlavB_cand2","Jet_btagDeepFlavB.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_particleNetAK4_B_cand1","Jet_particleNetAK4_B.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_particleNetAK4_B_cand2","Jet_particleNetAK4_B.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_cand1","Jet_HHbtag.at(bjet1_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_cand2","Jet_HHbtag.at(bjet2_JetIdx)")
    df[p] = df[p].Define("Jet_HHbtag_total","Jet_HHbtag.at(bjet1_JetIdx) + Jet_HHbtag.at(bjet2_JetIdx)")

#mycall_b1 = "index_bjet(5, GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"
#mycall_b2 = "index_bjet(-5, GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"

# INDICES GENERADORES
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet1","index_bjet_2(GenJet_partonFlavour, Jet_genJetIdx)[0]")
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2","index_bjet_2(GenJet_partonFlavour, Jet_genJetIdx)[1]")

Index1 = df["ggf_sm"].Histo1D(("Index_gen1","Index Gen b;Index Gen b;N_{Events}",32,-12,20),"idx_genbjet1","myweight")
Index2 = df["ggf_sm"].Histo1D(("Index_gen2","Index Gen anti b;Index Gen anti b;N_{Events}",32,-12,20),"idx_genbjet2","myweight")
Index_2D=df["ggf_sm"].Histo2D(("Index_2D","Index bjet 2D;Index jet1;Index jet2",32,-12,20,32,-12,20), "idx_genbjet1", "idx_genbjet2","myweight")


# COINCIDENCIAS CON HHBTAG
mycall_coinc_hhbtag = "index_coin_hhbtag(idx_genbjet1, idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_coinc_hhbtag",mycall_coinc_hhbtag)
Index_coin_hhgbtag = df["ggf_sm"].Histo1D(("Index_hhbtag_coin","Indexes coincidence hhbtag;Indexes coincidence hhbtag;N_{Events}",5,-2,3),"idx_coinc_hhbtag","myweight")    

# COINCIDENCIAS CON DEEPFLAVOUR
mycall_coinc_deepf = "index_coin_vec(idx_genbjet1, idx_genbjet2, Jet_btagDeepFlavB)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_coinc_deepf",mycall_coinc_deepf)
Index_coin_deepf = df["ggf_sm"].Histo1D(("Index_deepf_coin","Indexes coincidence deepf;Indexes coincidence deepf;N_{Events}",5,-2,3),"idx_coinc_deepf","myweight")    

# COINCIDENCIAS CON PARTICLE NET
mycall_coinc_ak4 = "index_coin_vec(idx_genbjet1, idx_genbjet2, Jet_particleNetAK4_B)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_coinc_ak4",mycall_coinc_ak4)
Index_coin_ak4 = df["ggf_sm"].Histo1D(("Index_ak4_coin","Indexes coincidence ak4;Indexes coincidence ak4;N_{Events}",5,-2,3),"idx_coinc_ak4","myweight")    

# Separo casos en los que hay solo uno de los jets a nivel generador
df_jets_1_solo = df["ggf_sm"].Filter("idx_genbjet1 == -11","Solo antib")
df_jets_1_solo = df_jets_1_solo.Filter("idx_genbjet2 >= 0","Solo indices positivos")
df_jets_2_solo = df["ggf_sm"].Filter("idx_genbjet2 == -11","Solo b")
df_jets_2_solo = df_jets_2_solo.Filter("idx_genbjet1 >= 0","Solo indices positivos")

df_jets_2_solo= df_jets_2_solo.Define("Jet_DeepFlavB_gen1","Jet_btagDeepFlavB.at(idx_genbjet1)")
df_jets_1_solo= df_jets_1_solo.Define("Jet_DeepFlavB_gen2","Jet_btagDeepFlavB.at(idx_genbjet2)")
df_jets_2_solo= df_jets_2_solo.Define("Jet_particleNetAK4_B_gen1","Jet_particleNetAK4_B.at(idx_genbjet1)")
df_jets_1_solo= df_jets_1_solo.Define("Jet_particleNetAK4_B_gen2","Jet_particleNetAK4_B.at(idx_genbjet2)")
df_jets_2_solo= df_jets_2_solo.Define("Jet_HHbtag_gen1","Jet_HHbtag.at(idx_genbjet1)")
df_jets_1_solo= df_jets_1_solo.Define("Jet_HHbtag_gen2","Jet_HHbtag.at(idx_genbjet2)")
# Miro las distribuciones que tengan en HHbtag, deepflav y ak4 y isBoosted
only1_deepf_2 = df_jets_2_solo.Histo1D(("DeepFlav_only1_b", "DeepFlavourB only1;DeepFlavB b;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen1","myweight")
only1_deepf_1 = df_jets_1_solo.Histo1D(("DeepFlav_only1_anti", "DeepFlavourB only1;DeepFlavB antib;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen2","myweight")
only1_ak4_2 = df_jets_2_solo.Histo1D(("AK4_only1_b", "particleNetAK4 only1;particleNetAK4 b;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen1","myweight")
only1_ak4_1 = df_jets_1_solo.Histo1D(("AK4_only1_antib", "particleNetAK4 only1;particleNetAK4 antib;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen2","myweight")
only1_hhbtag_2 = df_jets_2_solo.Histo1D(("HHbtagonly1_b", "HHbtag only1;HHbtag b;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen1","myweight")
only1_hhbtag_1 = df_jets_1_solo.Histo1D(("HHbtagonly1_antib", "HHbtag only1;HHbtag antib;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen2","myweight")
only1_boosted_2 = df_jets_2_solo.Histo1D(("boosted_only1_b", "boosted b only1;DeepFlavB b;N_{Events}",2,0, 2), "isBoosted","myweight")
only1_boosted_1 = df_jets_1_solo.Histo1D(("boosted_only1_anti", "boosted anti only1;DeepFlavB b;N_{Events}",2,0, 2), "isBoosted","myweight")




# Filtro índices negativos
df["ggf_sm"] = df["ggf_sm"].Filter("idx_genbjet1 >= 0", "Index 1 positive")
df["ggf_sm"] = df["ggf_sm"].Filter("idx_genbjet2 >= 0", "Index 2 positive")

mycall_order_pt = "index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)"
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet1_pt","index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)[0]")
df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2_pt","index_pt(idx_genbjet1, idx_genbjet2, Jet_pt)[1]")

#mycall_coinc = "index_coin(idx_genbjet1, idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx_coinc",mycall_coinc)

df["ggf_sm"] = df["ggf_sm"].Define("Jet_DeepFlavB_gen1","Jet_btagDeepFlavB.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_DeepFlavB_gen2","Jet_btagDeepFlavB.at(idx_genbjet2_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_particleNetAK4_B_gen1","Jet_particleNetAK4_B.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_particleNetAK4_B_gen2","Jet_particleNetAK4_B.at(idx_genbjet2_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_HHbtag_gen1","Jet_HHbtag.at(idx_genbjet1_pt)")
df["ggf_sm"] = df["ggf_sm"].Define("Jet_HHbtag_gen2","Jet_HHbtag.at(idx_genbjet2_pt)")

#mycall4 = "gen_phi2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_phi)"
#mycall4 = "gen_eta2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta)"
#mycall4 = "gen_index2(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_eta, Jet_eta, GenPart_phi, Jet_phi)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx_genbjet2",mycall4)
#mycall6 = "index1_coinc2(idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx2_coin",mycall6)

#mycall_tot = "index_tot(idx_genbjet1, idx_genbjet2, bjet1_JetIdx, bjet2_JetIdx)"
#df["ggf_sm"] = df["ggf_sm"].Define("idx_tot",mycall_tot)
    
DeepFlav_bjet1 = {}
DeepFlav_bjet2 = {}
AK4_bjet1 = {}
AK4_bjet2 = {}
HHbtag_bjet1 = {}
HHbtag_bjet2 = {}
HHbtag_total = {}
DeepFlav_2D = {}
AK4_2D = {}
HHbtag_2D = {}

for i in range(nProcesses):
    p=processes[i]
    DeepFlav_bjet1[p] = df[p].Histo1D(("DeepFlav_bjet1_{}".format(p), "DeepFlavourB Jet1;DeepFlavB jet1;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_cand1","myweight")
    DeepFlav_bjet2[p] = df[p].Histo1D(("DeepFlav_bjet2_{}".format(p), "DeepFlavourB Jet2;DeepFlavB jet2;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_cand2","myweight")
    AK4_bjet1[p] = df[p].Histo1D(("AK4_bjet1_{}".format(p), "particleNetAK4 Jet1;particleNetAK4 jet1;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_cand1","myweight")
    AK4_bjet2[p] = df[p].Histo1D(("AK4_bjet2_{}".format(p), "particleNetAK4 Jet2;particleNetAK4 jet2;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_cand2","myweight")
    HHbtag_bjet1[p] = df[p].Histo1D(("HHbtag_bjet1_{}".format(p), "HHbtag Jet1;HHbtag jet1;N_{Events}",60,0, 2.2), "Jet_HHbtag_cand1","myweight")
    HHbtag_bjet2[p] = df[p].Histo1D(("HHbtag_bjet2_{}".format(p), "HHbtag Jet2;HHbtag jet2;N_{Events}",60,0, 2.2), "Jet_HHbtag_cand2","myweight")
    HHbtag_total[p] = df[p].Histo1D(("HHbtag_total_{}".format(p), "HHbtag 2 jets;HHbtag 2 jets;N_{Events}",60,0, 2.2), "Jet_HHbtag_total","myweight")
    
    DeepFlav_2D[p] = df[p].Histo2D(("DeepFlavB_2D_{}".format(p),"Jet btagDeepFlav 2D;DeepFlavB jet1;DeepFlavB jet2",30,0,1.2,30,0,1.2), "Jet_DeepFlavB_cand1", "Jet_DeepFlavB_cand2","myweight")
    AK4_2D[p] = df[p].Histo2D(("AK4_2D_{}".format(p),"Jet particleNetAK4 2D;particleNetAK4 jet1;particleNetAK4 jet2",30,0,1.2,30,0,1.2), "Jet_particleNetAK4_B_cand1", "Jet_particleNetAK4_B_cand2","myweight")
    HHbtag_2D[p] = df[p].Histo2D(("HHbtag_2D_{}".format(p),"Jet HHbtag 2D;HHbtag jet1;HHbtag jet2",60,0,2.2,60,0,2.2), "Jet_HHbtag_cand1", "Jet_HHbtag_cand2","myweight")

    
#Gen_phi1 = df["ggf_sm"].Histo1D(("Gen_phi1","Phi Gen b;Phi Gen b;N_{Events}",30,-5,11),"Genb1_phi","myweight")
#Gen_phi2 = df["ggf_sm"].Histo1D(("Gen_phi2","Phi Gen anti b;Phi Gen anyi b;N_{Events}",30,-5,11),"Genb2_phi","myweight")

#Index1_coin = df["ggf_sm"].Histo1D(("Index_gen1_coin","Index 1 coincidence;Index 1 coincidence;N_{Events}",5,-2,3),"idx1_coin","myweight")
#Index2_coin = df["ggf_sm"].Histo1D(("Index_gen2_coin","Index 2 coincidence;Index 2 coincidence;N_{Events}",5,-2,3),"idx2_coin","myweight")
#Index_2D = df["ggf_sm"].Histo2D(("Index_2D","Index bjet 2D;Index jet1;Index jet2",5,-2,3,5,-2,3), "idx1_coin", "idx2_coin","myweight")
#Index_tot = df["ggf_sm"].Histo1D(("Index_tot_coin","Indexes coincidence;Indexes coincidence;N_{Events}",5,-2,3),"idx_coinc","myweight")    

DeepFlav_bjet1_gen = df["ggf_sm"].Histo1D(("DeepFlav_bjet1_gen", "DeepFlavourB Jet1;DeepFlavB jet1;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen1","myweight")
DeepFlav_bjet2_gen = df["ggf_sm"].Histo1D(("DeepFlav_bjet2_gen", "DeepFlavourB Jet2;DeepFlavB jet2;N_{Events}",30,0, 1.2), "Jet_DeepFlavB_gen2","myweight")
AK4_bjet1_gen = df["ggf_sm"].Histo1D(("AK4_bjet1_gen", "particleNetAK4 Jet1;particleNetAK4 jet1;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen1","myweight")
AK4_bjet2_gen = df["ggf_sm"].Histo1D(("AK4_bjet2_gen", "particleNetAK4 Jet2;particleNetAK4 jet2;N_{Events}",30,0, 1.2), "Jet_particleNetAK4_B_gen2","myweight")
HHbtag_bjet1_gen = df["ggf_sm"].Histo1D(("HHbtag_bjet1_gen", "HHbtag Jet1;HHbtag jet1;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen1","myweight")
HHbtag_bjet2_gen = df["ggf_sm"].Histo1D(("HHbtag_bjet2_gen", "HHbtag Jet2;HHbtag jet2;N_{Events}",60,0, 2.2), "Jet_HHbtag_gen2","myweight")
DeepFlav_2D_gen = df["ggf_sm"].Histo2D(("DeepFlavB_2D_gen","Jet btagDeepFlav 2D;DeepFlavB jet1;DeepFlavB jet2",30,0,1.2,30,0,1.2), "Jet_DeepFlavB_gen1", "Jet_DeepFlavB_gen2","myweight")
AK4_2D_gen = df["ggf_sm"].Histo2D(("AK4_2D_gen","Jet particleNetAK4 2D;particleNetAK4 jet1;particleNetAK4 jet2",30,0,1.2,30,0,1.2), "Jet_particleNetAK4_B_gen1", "Jet_particleNetAK4_B_gen2","myweight")
HHbtag_2D_gen = df["ggf_sm"].Histo2D(("HHbtag_2D_gen","Jet HHbtag 2D;HHbtag jet1;HHbtag jet2",60,0,2.2,60,0,2.2), "Jet_HHbtag_gen1", "Jet_HHbtag_gen2","myweight")


out = ROOT.TFile("histos_indexes.root","RECREATE")
out.cd()

for i in range(nProcesses):
    p = processes[i]
    
    DeepFlav_bjet1[p].Write()
    DeepFlav_bjet2[p].Write()
    AK4_bjet1[p].Write()
    AK4_bjet2[p].Write()
    HHbtag_bjet1[p].Write()
    HHbtag_bjet2[p].Write()
    HHbtag_total[p].Write()
    DeepFlav_2D[p].Write()
    AK4_2D[p].Write()
    HHbtag_2D[p].Write()
    
#Gen_phi1.Write()
#Gen_phi2.Write()
Index1.Write()
Index2.Write()
Index_2D.Write()
#Index1_coin.Write()
#Index2_coin.Write()
#Index_2D.Write()
Index_coin_hhgbtag.Write()
Index_coin_deepf.Write()
Index_coin_ak4.Write()

DeepFlav_bjet1_gen.Write()
DeepFlav_bjet2_gen.Write()
AK4_bjet1_gen.Write()
AK4_bjet2_gen.Write()
HHbtag_bjet1_gen.Write()
HHbtag_bjet2_gen.Write()
DeepFlav_2D_gen.Write()
AK4_2D_gen.Write()
HHbtag_2D_gen.Write()


only1_deepf_1.Write()
only1_deepf_2.Write()
only1_ak4_1.Write()
only1_ak4_2.Write()
only1_hhbtag_1.Write()
only1_hhbtag_2.Write()
only1_boosted_1.Write()
only1_boosted_2.Write()

out.Close()
print("Terminado")

Reading Trees
['ggf_sm', 'tt_dl', 'tt_sl', 'tt_fh', 'dy']
CutFlow for process ggf_sm
CutFlow for process tt_dl
CutFlow for process tt_sl
CutFlow for process tt_fh
CutFlow for process dy
Terminado


In [27]:
file=ROOT.TFile("histos_indexes.root","READONLY")
luminosity = 58000 # pb

histos2D_i = ["Index_hhbtag_coin"]
h= file.Get("Index_hhbtag_coin")
h.SetName("h"+"Index_hhbtag_coin")
h.Scale(luminosity)
    
c3 = ROOT.TCanvas("c3","HH analysis canvas", 600,800)
ROOT.gStyle.SetOptTitle(True)

c3.SetTicks(1,1)
c3.SetLeftMargin(0.15)
c3.SetRightMargin(0.08)
c3.SetTopMargin(0.06)
#c3.SetLogz()
h.Draw("hist")
xlabel=h.GetXaxis().GetTitle()
ylabel=h.GetYaxis().GetTitle()
h.GetXaxis().SetTitle(xlabel)
h.GetYaxis().SetTitle(ylabel)
h.GetYaxis().SetTitleOffset(1.2)
h.GetXaxis().SetTitleOffset(1.2)
title = "Index_hhbtag_coin"
h.SetTitle(title)
#h.GetXaxis().SetRangeUser(-2,3)
#h.GetYaxis().SetRangeUser(-2,3)
#h.SetStats(0)
print (h.GetXaxis().GetTitle() )

c3.SaveAs("Plot_"+"Index_hhbtag_coin"+".png")

indices1 = h.Integral(3,4)
indices2 = h.Integral(1,2)
total = h.Integral(1,4)
    
file.Close()

print(indices1)
print(indices2)
print(total)
print(indices1 / total * 100)

Indexes coincidence hhbtag
20.305147500000338
33.89293800000072
54.198085500001056
37.46469513208163


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file Plot_Index_hhbtag_coin.png has been created


In [28]:
file=ROOT.TFile("histos_indexes.root","READONLY")
luminosity = 58000 # pb

histos2D_i = ["Index_deepf_coin"]
h= file.Get("Index_deepf_coin")
h.SetName("h"+"Index_deepf_coin")
h.Scale(luminosity)
    
c3 = ROOT.TCanvas("c3","HH analysis canvas", 600,800)
ROOT.gStyle.SetOptTitle(True)

c3.SetTicks(1,1)
c3.SetLeftMargin(0.15)
c3.SetRightMargin(0.08)
c3.SetTopMargin(0.06)
#c3.SetLogz()
h.Draw("hist")
xlabel=h.GetXaxis().GetTitle()
ylabel=h.GetYaxis().GetTitle()
h.GetXaxis().SetTitle(xlabel)
h.GetYaxis().SetTitle(ylabel)
h.GetYaxis().SetTitleOffset(1.2)
h.GetXaxis().SetTitleOffset(1.2)
title = "Index_deepf_coin"
h.SetTitle(title)
#h.GetXaxis().SetRangeUser(-2,3)
#h.GetYaxis().SetRangeUser(-2,3)
#h.SetStats(0)
print (h.GetXaxis().GetTitle() )

c3.SaveAs("Plot_"+"Index_deepf_coin"+".png")

indices1 = h.Integral(3,4)
indices2 = h.Integral(1,2)
total = h.Integral(1,4)
    
file.Close()

print(indices1)
print(indices2)
print(total)
print(indices1 / total * 100)

Indexes coincidence deepf
25.793390250000517
28.404695250000604
54.19808550000112
47.590961953812894


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file Plot_Index_deepf_coin.png has been created


In [29]:
file=ROOT.TFile("histos_indexes.root","READONLY")
luminosity = 58000 # pb

histos2D_i = ["Index_ak4_coin"]
h= file.Get("Index_ak4_coin")
h.SetName("h"+"Index_ak4_coin")
h.Scale(luminosity)
    
c3 = ROOT.TCanvas("c3","HH analysis canvas", 600,800)
ROOT.gStyle.SetOptTitle(True)

c3.SetTicks(1,1)
c3.SetLeftMargin(0.15)
c3.SetRightMargin(0.08)
c3.SetTopMargin(0.06)
#c3.SetLogz()
h.Draw("hist")
xlabel=h.GetXaxis().GetTitle()
ylabel=h.GetYaxis().GetTitle()
h.GetXaxis().SetTitle(xlabel)
h.GetYaxis().SetTitle(ylabel)
h.GetYaxis().SetTitleOffset(1.2)
h.GetXaxis().SetTitleOffset(1.2)
title = "Index_ak4_coin"
h.SetTitle(title)
#h.GetXaxis().SetRangeUser(-2,3)
#h.GetYaxis().SetRangeUser(-2,3)
#h.SetStats(0)
print (h.GetXaxis().GetTitle() )

c3.SaveAs("Plot_"+"Index_ak4_coin"+".png")

indices1 = h.Integral(3,4)
indices2 = h.Integral(1,2)
total = h.Integral(1,4)
    
file.Close()

print(indices1)
print(indices2)
print(total)
print(indices1 / total * 100)

Indexes coincidence ak4
28.80089325000061
25.397192250000504
54.19808550000111
53.1400564877887


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Info in <TCanvas::Print>: png file Plot_Index_ak4_coin.png has been created


In [8]:
# ESTE ES EL SCRIPT DE TDRSTYLE
# ==================================================
# ==================================================

import ROOT as rt

def tdrGrid( gridOn):
    tdrStyle.SetPadGridX(gridOn)
    tdrStyle.SetPadGridY(gridOn)

#fixOverlay: Redraws the axis
def fixOverlay(): gPad.RedrawAxis()

def setTDRStyle():
    tdrStyle =  rt.TStyle("tdrStyle","Style for P-TDR")

   #for the canvas:
    tdrStyle.SetCanvasBorderMode(0)
    tdrStyle.SetCanvasColor(rt.kWhite)
    tdrStyle.SetCanvasDefH(600) #Height of canvas
    tdrStyle.SetCanvasDefW(600) #Width of canvas
    tdrStyle.SetCanvasDefX(0)   #POsition on screen
    tdrStyle.SetCanvasDefY(0)


    tdrStyle.SetPadBorderMode(0)
  #tdrStyle.SetPadBorderSize(Width_t size = 1)
    tdrStyle.SetPadColor(rt.kWhite)
#  tdrStyle.SetPadGridX(True)
#  tdrStyle.SetPadGridY(True)
    tdrStyle.SetGridColor(0)
    tdrStyle.SetGridStyle(3)
    tdrStyle.SetGridWidth(1)

#For the frame:
    tdrStyle.SetFrameBorderMode(0)
    tdrStyle.SetFrameBorderSize(1)
    tdrStyle.SetFrameFillColor(0)
    tdrStyle.SetFrameFillStyle(0)
    tdrStyle.SetFrameLineColor(1)
    tdrStyle.SetFrameLineStyle(1)
    tdrStyle.SetFrameLineWidth(1)

#For the histo:
    #tdrStyle.SetHistFillColor(1)
    #tdrStyle.SetHistFillStyle(0)
    tdrStyle.SetHistLineColor(1)
    tdrStyle.SetHistLineStyle(0)
    tdrStyle.SetHistLineWidth(1)
    #tdrStyle.SetLegoInnerR(Float_t rad = 0.5)
    #tdrStyle.SetNumberContours(Int_t number = 20)

    tdrStyle.SetEndErrorSize(2)
    #tdrStyle.SetErrorMarker(20)
    #tdrStyle.SetErrorX(0.)
  
    tdrStyle.SetMarkerStyle(20)
    
#For the fit/function:
    tdrStyle.SetOptFit(1)
    tdrStyle.SetFitFormat("5.4g")
    tdrStyle.SetFuncColor(2)
    tdrStyle.SetFuncStyle(1)
    tdrStyle.SetFuncWidth(1)

#For the date:
    tdrStyle.SetOptDate(0)
    # tdrStyle.SetDateX(Float_t x = 0.01)
    # tdrStyle.SetDateY(Float_t y = 0.01)

# For the statistics box:
    tdrStyle.SetOptFile(0)
    tdrStyle.SetOptStat("")#mroi") # To display the mean and RMS:   SetOptStat("mr")
    tdrStyle.SetStatColor(rt.kWhite)
    tdrStyle.SetStatFont(42)
    tdrStyle.SetStatFontSize(0.025)
    tdrStyle.SetStatTextColor(1)
    tdrStyle.SetStatFormat("6.4g")
    tdrStyle.SetStatBorderSize(1)
    tdrStyle.SetStatH(0.1)
    tdrStyle.SetStatW(0.15)
    # tdrStyle.SetStatStyle(Style_t style = 1001)
    # tdrStyle.SetStatX(Float_t x = 0)
    # tdrStyle.SetStatY(Float_t y = 0)

# Margins:
    tdrStyle.SetPadTopMargin(0.05)
    tdrStyle.SetPadBottomMargin(0.13)
    tdrStyle.SetPadLeftMargin(0.16)
    tdrStyle.SetPadRightMargin(0.02)

# For the Global title:

    tdrStyle.SetOptTitle(0)
    tdrStyle.SetTitleFont(42)
    tdrStyle.SetTitleColor(1)
    tdrStyle.SetTitleTextColor(1)
    tdrStyle.SetTitleFillColor(10)
    tdrStyle.SetTitleFontSize(0.05)
    # tdrStyle.SetTitleH(0) # Set the height of the title box
    # tdrStyle.SetTitleW(0) # Set the width of the title box
    # tdrStyle.SetTitleX(0) # Set the position of the title box
    # tdrStyle.SetTitleY(0.985) # Set the position of the title box
    # tdrStyle.SetTitleStyle(Style_t style = 1001)
    # tdrStyle.SetTitleBorderSize(2)

# For the axis titles:

    tdrStyle.SetTitleColor(1, "XYZ")
    tdrStyle.SetTitleFont(42, "XYZ")
    tdrStyle.SetTitleSize(0.04, "XYZ")
    # tdrStyle.SetTitleXSize(Float_t size = 0.02) # Another way to set the size?
    # tdrStyle.SetTitleYSize(Float_t size = 0.02)
    tdrStyle.SetTitleXOffset(0.9)
    tdrStyle.SetTitleYOffset(1.25)
    # tdrStyle.SetTitleOffset(1.1, "Y") # Another way to set the Offset


# For the axis labels:

    tdrStyle.SetLabelColor(1, "XYZ")
    tdrStyle.SetLabelFont(42, "XYZ")
    tdrStyle.SetLabelOffset(0.007, "XYZ")
    tdrStyle.SetLabelSize(0.04, "XYZ")

# For the axis:

    tdrStyle.SetAxisColor(1, "XYZ")
    tdrStyle.SetStripDecimals(True)
    tdrStyle.SetTickLength(0.03, "XYZ")
    tdrStyle.SetNdivisions(510, "XYZ")
    tdrStyle.SetPadTickX(1)  # To get tick marks on the opposite side of the frame
    tdrStyle.SetPadTickY(1)

# Change for log plots:
    tdrStyle.SetOptLogx(0)
    tdrStyle.SetOptLogy(0)
    tdrStyle.SetOptLogz(0)

# Postscript options:
    tdrStyle.SetPaperSize(20.,20.)
    # tdrStyle.SetLineScalePS(Float_t scale = 3)
    # tdrStyle.SetLineStyleString(Int_t i, const char* text)
    # tdrStyle.SetHeaderPS(const char* header)
    # tdrStyle.SetTitlePS(const char* pstitle)

    # tdrStyle.SetBarOffset(Float_t baroff = 0.5)
    # tdrStyle.SetBarWidth(Float_t barwidth = 0.5)
    # tdrStyle.SetPaintTextFormat(const char* format = "g")
    # tdrStyle.SetPalette(Int_t ncolors = 0, Int_t* colors = 0)
    # tdrStyle.SetTimeOffset(Double_t toffset)
    # tdrStyle.SetHistMinimumZero(kTRUE)

    tdrStyle.SetHatchesLineWidth(5)
    tdrStyle.SetHatchesSpacing(0.05)

    tdrStyle.cd()